rmt lk 184 kuni 187.
Näidiskood: https://github.com/PacktPublishing/TinyML-Cookbook/blob/main/Chapter05/ColabNotebooks/prepare_model.ipynb

# Train model

Onen in Colab and upload dataset.zip.

0. Import libraries

In [ ]:
import numpy as np
import pathlib
import tensorflow as tf
import tensorflow_datasets as tfds
import zipfile
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.models import Model

1. Constants

In [ ]:
MODEL_ALPHA = 0.35
MODEL_INPUT_WIDTH = 48
MODEL_INPUT_HEIGHT = 48
TFL_MODEL_FILE = "scene_recognition.tflite"
TFL_MODEL_HEADER_FILE = "scene_recognition_model.h"
TF_MODEL = "scene_recognition"

## Transfer learning with Keras


2. Unzip the dataset:

In [ ]:
with zipfile.ZipFile("dataset.zip", 'r') as zip_ref:
  zip_ref.extractall(".")

data_dir = "dataset"

3. Resizes the input images to 48x48 with the bilinear interpolation. Prepare the training (80%) and validation (20%) datsets:

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  interpolation="bilinear",
  image_size=(MODEL_INPUT_WIDTH, MODEL_INPUT_HEIGHT)) # const

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  interpolation="bilinear",
  image_size=(MODEL_INPUT_WIDTH, MODEL_INPUT_HEIGHT))

4. Get the name of the classes

In [ ]:
class_names = train_ds.class_names
num_classes = len(class_names)
print(class_names)

5. Rescale the pixel values from [0, 255] to [-1, 1] because the pre-trained model expects this interval data range for the input tensor.

In [ ]:
rescale = tf.keras.layers.Rescaling(1./255, offset= -1)
train_ds = train_ds.map(lambda x, y: (rescale(x), y))
val_ds = val_ds.map(lambda x, y: (rescale(x), y))

6. Import the MobileNet v2 pre-trained model with the weights trained on the ImageNet dataset and alpha=0.35. Furthermore, set the input image at the lowest resolution allowed by the pre-trained model (48, 48, 3) and exclude the top (fully-connected) layers:

In [ ]:
base_model = MobileNetV2(input_shape=(48, 48, 3),
                         include_top=False,
                         weights='imagenet',
                         alpha=0.35)

7. Freeze the weights so that you do not update these values during training:

In [ ]:
base_model.trainable = False
feat_extr = base_model

8. Augment the input data:

In [ ]:
augmen = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

train_ds = train_ds.map(lambda x, y: (augmen(x), y))
val_ds = val_ds.map(lambda x, y: (augmen(x), y))

9. Prepare the classification head with a global pooling followed by a dense layer with a softmax activation:

In [ ]:
global_avg_layer = tf.keras.layers.GlobalAveragePooling2D()

dense_layer = tf.keras.layers.Dense(num_classes,activation='softmax')

10. Build the model architecture:

In [ ]:
inputs = tf.keras.Input(shape=(MODEL_INPUT_WIDTH, MODEL_INPUT_HEIGHT, 3))
x = global_avg_layer(feat_extr.layers[-1].output)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = dense_layer(x)
model = tf.keras.Model(inputs=feat_extr.inputs,outputs=outputs)

11. Compile the model with a 0.0005 learning rate:

In [ ]:
lr = 0.0005

model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
  metrics=['accuracy'])

12. Model summary

In [ ]:
model.summary()

13. Train the model with 10 epochs:

In [ ]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10)

14.  Save the TensorFlow model:

In [ ]:
model.save(TF_MODEL)